In [1]:
# If the libraries are not yet installed, they can be installed in this notebook using commands similar to the below
# %conda install numpy
# %conda install pandas
# %conda install matplotlib
# %conda install scikit-learn
# %conda install -c conda-forge lightgbm 
# %conda install -c conda-forge bayesian-optimization 
# %conda install -c conda-forge scipy
# %conda install joblib
# %conda install tdqm

# Something like the following may also work if the above does not
# import sys
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} scikit-learn
# !conda install -c conda-forge --yes --prefix {sys.prefix} lightgbm
# !conda install -c conda-forge --yes --prefix {sys.prefix} bayesian-optimization 
# !conda install -c conda-forge --yes --prefix {sys.prefix} scipy 
# !conda install --yes --prefix {sys.prefix} joblib
# !conda install --yes --prefix {sys.prefix} tdqm

# To install a specific version, add the version to the install command
# E.g., %conda install numpy=1.20.3

# If all else fails, use pip or follow additional advice such as found at
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

# If your plan to use pip (especially if you are not working within a specified conda environment), 
# the pip commands might look like:
# pip install numpy
# pip install pandas
# pip install scikit-learn
# pip install lightgbm
# pip install bayesian-optimization 
# pip install scipy
# pip install joblib
# pip install tdqm

# To install a specific version, add the version to the pip install command
# E.g., pip install numpy==1.20.3

In [2]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import glob
from lightgbm import LGBMRegressor
import random
from sklearn.model_selection import ParameterSampler
import scipy
import gc
from joblib import Parallel, delayed
import contextlib
import joblib
from tqdm import tqdm
from bayes_opt import BayesianOptimization

In [3]:
np.random.seed(54321)
random.seed(54321)

# Read in Data and Prepare for Modeling

In [4]:
# Create an empty list to hold the dataframes of highways england data
total_df_list = list()

# Loop through the files, sorted in alphabetical order
# Read them into a df, make sure they are sorted by timestamp, and append to the list
for fname in sorted(glob.glob("Data/Unseen Sensor/Processed/*.csv")):
    print("Reading {}".format(fname))
    df = pd.read_csv(fname) #, parse_dates=['timestamp'], index_col=['timestamp'])
    df = df.sort_values(by="timestamp")
    total_df_list.append(df)

Reading Data/Unseen Sensor/Processed/A19-9336-1_Northbound_2019_Processed.csv
Reading Data/Unseen Sensor/Processed/A66-9521-1_Westbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M40-7048-2_Southbound_Processed.csv
Reading Data/Unseen Sensor/Processed/M62-2056A_Eastbound_Processed.csv


In [5]:
# Read in the start and end points csv, and subtract 1 to deal with index differences between R and python
start_end = pd.read_csv("start_end_points_unseen.csv")
start_end["start"] = start_end["start"] - 1
start_end["end"] = start_end["end"]

In [6]:
# Create an empty list to hold the subset data frames (those with only 12 weeks of data per highway)
subset_df_list = list()

In [7]:
# For each df in our original total df list
for idx, df in enumerate(total_df_list):
        
    # Filter the timeframe based on the start_end_points csv files
    subset_df = df.iloc[start_end.iloc[idx,0]:start_end.iloc[idx,1], ]\
    .reset_index(drop=True).reset_index(drop=False)\
    .rename(columns={"index":"rn"})
    
    # Create a new field called train_val_test to differentiate each set of data
    subset_df["train_val_test"] = np.where(subset_df["rn"]<(96*7*8),
                                           "train",
                                           np.where(subset_df["rn"]<(96*7*10),
                                                    "val",
                                                    "test"
                                                   )
                                       )
    
    # Append to list
    subset_df_list.append(subset_df)

In [8]:
# Create a list of df's with only fields we need

# Initialize empty list
model_df_list = list()

# For df in subset list
for df in subset_df_list:
       
    # Extract the timestamp, the volume, and the train_val_test assignment
    model_df = df[['timestamp', 'total_volume', "train_val_test"]]\
    .rename(columns={'timestamp':'start', 'total_volume':'target'})
    
    # Append this df to the new list
    model_df_list.append(model_df)

## Create Lag Emebedded Matrices for each TS

In [9]:
lag_n = 840

In [10]:
# # Lag embed the data frames and save to a list
lag_embed_df_list = list()

for df in model_df_list:
    # For each df in our list
    for n in range(1, (lag_n+1)):
        # For each lag level, up to 960
        # Create a new column called target-n
        name = f"target-{n}"
        # Save the target shifted n values into this colume
        df[name] = df['target'].shift(n)
    # Append to list
    lag_embed_df_list.append(df)

/tmp/ipykernel_5801/3230010149.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[name] = df['target'].shift(n)


In [11]:
# Split the lag embedded list into train, val, and test lists

# First, initialize empty lists for each train, val, and test
train_df_list = list()
val_df_list = list()
test_df_list = list()

for i in range(len(lag_embed_df_list)):
    # For each df in our list
    df = lag_embed_df_list[i].copy()

    # Add a ts_index of i+1 to join with clustering data from R
    df['ts_index'] = i + 1
    
    # Subset into train, val, and test df's based on the train_val_test_field
    train_df = df.query("train_val_test == 'train'").copy()
    val_df = df.query("train_val_test=='val'").copy()
    test_df = df.query("train_val_test=='test'").copy()
    
    # Append to appropriate lists
    train_df_list.append(train_df)
    val_df_list.append(val_df)
    test_df_list.append(test_df)

In [12]:
# Concat all dfs from the lists together to create one full train, val, and test df
train_df_full = pd.concat(train_df_list)
val_df_full = pd.concat(val_df_list)
test_df_full = pd.concat(test_df_list)

In [13]:
# Drop unneeded columns
train_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
val_df_full.drop(columns=['start', 'train_val_test'], inplace=True)
test_df_full.drop(columns=['start', 'train_val_test'], inplace=True)

In [14]:
# Append the training and validation data together for later use
train_val_df_full = train_df_full.append(val_df_full)

In [15]:
# Delete unused variables to free up memory
del train_df_list
del val_df_list 
del test_df_list
del lag_embed_df_list
del model_df_list
del subset_df_list

In [16]:
# Force garbage collection to free up memory
gc.collect()

0

In [17]:
unseen_sen_clust = pd.read_csv("Results/Unseen Sensor/clust_assign.csv")

In [18]:
unseen_sen_clust.head()

,Unnamed: 0,rand,catch22,tsfeat,dtw
0,1,3,1,1,2
1,2,2,1,1,2
2,3,1,4,1,1
3,4,4,4,1,1


# Full Model

In [19]:
# Save model to file to use later
filename = 'Results/Global/LightGBM Bayes/Full/model'
mod = joblib.load(filename)

In [20]:
res = pd.read_csv('Results/Global/LightGBM Bayes/Full/residual.csv')

In [21]:
# Function to compute test preds
def compute_lgbm_test_preds(mod, data, lag_n):
    """Function which takes in: a model, test data, and the lag embedding to use, and returns a df of forecasts"""

    # Initialize an empty data frame to store preds
    pred_df = pd.DataFrame()
    
    # Loop through each individual time series index in the data set
    for ts_idx in data.ts_index.unique():
        # Create the X matrix for each one
        X = data.query("ts_index==@ts_idx").iloc[:,1:(lag_n+1)].copy()

        # Forecast for that X matrix
        preds = mod.predict(X)
        
        # Save the results to a temp data frame
        pred_df_sub = pd.DataFrame({"ts_index": ts_idx, "test_preds": preds})
        
        # Append to primary data frame
        pred_df = pred_df.append(pred_df_sub)
    
    # Return df of all preds with corresponding ts_index column
    return pred_df

In [22]:
unseen_test_preds = compute_lgbm_test_preds(mod, test_df_full, lag_n)

In [23]:
# Function to compute performance metrics on test data
def compute_lgbm_test_perf(preds, data):
    """Function which takes inputs: a data frame of test predictions, and a test data df,
    and which returns a data frame of model performance"""
    
    # Create an empty list to store model performance
    perf_ls = list()
    
    # For each time series index in our data set
    for ts_idx in data.ts_index.unique():
        # Get the target (actual) for that index
        y_sub = data.query("ts_index==@ts_idx").iloc[:,0]
        # Extract the corresponding forecasts
        preds_sub = preds.query("ts_index==@ts_idx").test_preds
        
        # Compute rmse, mae, and the mean of the true target value for those preds
        rmse_sub = mean_squared_error(y_sub, preds_sub, squared=False)
        mae_sub = mean_absolute_error(y_sub, preds_sub)
        mean_sub = np.mean(y_sub)
        
        # Save those metrics to a dictionary
        pred_dict = {"rmse": rmse_sub, "mae": mae_sub, "mean": mean_sub}
        
        # Append the dictionary to the list
        perf_ls.append(pred_dict)
        
    # Return a data frame of model performance created from the list of dictionaries
    return pd.DataFrame(perf_ls)

In [24]:
# Compute model perf metrics using above function
full_mod_test_perf = compute_lgbm_test_perf(unseen_test_preds, test_df_full)

# Compute scaled performance metrics in new columns
full_mod_test_perf['nrmse'] = full_mod_test_perf['rmse']/full_mod_test_perf['mean']
full_mod_test_perf['smae'] = full_mod_test_perf['mae']/full_mod_test_perf['mean']

In [25]:
# Print the means of model perf metrics
full_mod_test_perf.mean()

rmse      38.058841
mae       25.287789
mean     377.058594
nrmse      0.122584
smae       0.082148
dtype: float64

In [26]:
# Function to compute pred intervals with bootstrap method
def compute_lgbm_boostrap_int(preds, resid, n_boot):
    """Function which takes in a model's predictions and residuals, and a number of bootstrap resamples to use,
    and which outputs a df with pred intervals at 80% and 95%"""
    
    # Set seeds for reproducibility
    random.seed(54321)
    np.random.seed(54321)
    
    # Create empty columns in the pred df to store the PIs
    preds['lo_95'] = np.nan
    preds['hi_95'] = np.nan
    preds['lo_80'] = np.nan
    preds['hi_80'] = np.nan
    
    # For each row in the pred df
    for n in range(preds.shape[0]):
        # Sample with replacement n_boot times from the residuals
        resid_boot = np.random.choice(resid, size=n_boot, replace=True)
        # Extract the forecast value for that row
        pred_n = preds.iloc[n, :].test_preds
        # Add the residual vector to the forecast value
        pred_n_boot = resid_boot + pred_n
        
        # Compute quantiles of this residual+forecast vector
        percent_95_lo = np.percentile(pred_n_boot, 2.5)
        percent_95_hi = np.percentile(pred_n_boot, 97.5)
        
        percent_80_lo = np.percentile(pred_n_boot, 10)
        percent_80_hi = np.percentile(pred_n_boot, 90)
        
        # Save these quantiles to the appropriate df column
        preds.iloc[n, 2] = percent_95_lo
        preds.iloc[n, 3] = percent_95_hi
        preds.iloc[n, 4] = percent_80_lo
        preds.iloc[n, 5] = percent_80_hi
    
    # Return the updated preds data frame
    return preds

In [27]:
# Compute PIs with 1000 bootstrap samples
nboot = 1000
full_mod_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds, 
                                               res.residual.values, 
                                               nboot)

In [28]:
# Add the true values into their own df column
full_mod_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [29]:
full_mod_boot_ints.head()

,ts_index,test_preds,lo_95,hi_95,lo_80,hi_80,actual
0,1,246.485289,195.983314,305.558795,222.563419,275.585662,221.0
1,1,240.022395,184.141914,302.085650,210.253851,267.699421,323.0
2,1,283.329092,225.566530,347.911164,253.318823,309.824978,298.0
3,1,289.137922,228.730568,349.433583,261.975540,317.719134,305.0
4,1,287.392703,227.063650,345.162505,256.256476,315.235074,238.0


In [30]:
# Create a function to compute the interval score
def interval_score(true_values, lower, upper, interval_range):
    """ Function which takes in the true values, the upper and lower bounds of PIs, and the PI level (e.g., 90%)
        and from these inputs, computes the interval score for each prediction
    """
    
    # Compute alpha from the interval range
    alpha = 1-interval_range
    
    # Save the upper, lower, and true_values as numpy arrays for computation purposes
    upper = np.array(upper)
    lower = np.array(lower)
    true_values = np.array(true_values)
    
    # Compute the lower component of the interval score - just a boolean for true below interval
    def lower_ind(true,low):
        if true<low:
            return 1
        else:
            return 0
        
    # Computer the upper component of the interval score - similar boolean for true above interval
    def upper_ind(true,up):
        if true>up:
            return 1
        else:
            return 0
        
    # Computer the actual score for each obsveration - formula here: https://epiforecasts.io/scoringutils/reference/interval_score.html
    scores = (upper-lower) + (2/alpha)*(lower-true_values)*(lower > true_values) + (2/alpha)*(true_values-upper)*(true_values > upper)
    
    # Return the scores array
    return scores

In [31]:
# Compute the 95% and 80% PI scores using the above function as new data frame columns
full_mod_boot_ints['int_95_score'] = interval_score(full_mod_boot_ints.actual, 
                                                    full_mod_boot_ints.lo_95,
                                                    full_mod_boot_ints.hi_95,
                                                    0.95)
                                                    
full_mod_boot_ints['int_80_score'] = interval_score(full_mod_boot_ints.actual, 
                                                    full_mod_boot_ints.lo_80,
                                                    full_mod_boot_ints.hi_80,
                                                    0.80)

In [32]:
# Print the means of the interval scores
full_mod_boot_ints.mean()

ts_index          2.500000
test_preds      377.539545
lo_95           316.929484
hi_95           440.369775
lo_80           348.905650
hi_80           407.135720
actual          377.058594
int_95_score    306.083380
int_80_score    157.330588
dtype: float64

In [33]:
full_mod_boot_ints_group = full_mod_boot_ints.groupby("ts_index").mean().reset_index()
full_mod_boot_ints_group['mean'] = full_mod_test_perf['mean'].values
full_mod_boot_ints_group['int_95_score_scaled'] = full_mod_boot_ints_group['int_95_score']/full_mod_boot_ints_group['mean']
full_mod_boot_ints_group['int_80_score_scaled'] = full_mod_boot_ints_group['int_80_score']/full_mod_boot_ints_group['mean']

In [34]:
full_mod_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.882222
int_80_score_scaled    0.488217
dtype: float64

# Random Clusters

In [35]:
unseen_sen_clust.rand

0    3
1    2
2    1
3    4
Name: rand, dtype: int64

In [36]:
rand_clust_mod3 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_2")
rand_clust_mod2 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_1")
rand_clust_mod1 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_0")
rand_clust_mod4 = joblib.load("Results/Global/LightGBM Bayes/Random Cluster/model_3")

In [37]:
rand_clust_res = pd.read_csv("Results/Global/LightGBM Bayes/Random Cluster/residual.csv")
rand_clust_res['residual'] = rand_clust_res['residual'].apply(eval)

res_rand_clust_3 = rand_clust_res.query("cluster==3")['residual'].values[0]
res_rand_clust_2 = rand_clust_res.query("cluster==2")['residual'].values[0]
res_rand_clust_1 = rand_clust_res.query("cluster==1")['residual'].values[0]
res_rand_clust_4 = rand_clust_res.query("cluster==4")['residual'].values[0]

In [38]:
unseen_test_preds_rand_clust3 = compute_lgbm_test_preds(rand_clust_mod3, 
                                                        test_df_full.query("ts_index==1"), 
                                                        lag_n)

unseen_test_preds_rand_clust2 = compute_lgbm_test_preds(rand_clust_mod2, 
                                                        test_df_full.query("ts_index==2"), 
                                                        lag_n)

unseen_test_preds_rand_clust1 = compute_lgbm_test_preds(rand_clust_mod1, 
                                                        test_df_full.query("ts_index==3"), 
                                                        lag_n)

unseen_test_preds_rand_clust4 = compute_lgbm_test_preds(rand_clust_mod4, 
                                                        test_df_full.query("ts_index==4"), 
                                                        lag_n)

In [39]:
# Compute model perf metrics using above function
rand_clust3_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust3, test_df_full.query("ts_index==1"))
rand_clust2_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust2, test_df_full.query("ts_index==2"))
rand_clust1_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust1, test_df_full.query("ts_index==3"))
rand_clust4_test_perf = compute_lgbm_test_perf(unseen_test_preds_rand_clust4, test_df_full.query("ts_index==4"))

rand_clust_test_perf = rand_clust3_test_perf.append(rand_clust2_test_perf).append(rand_clust1_test_perf).append(rand_clust4_test_perf)

# Compute scaled performance metrics in new columns
rand_clust_test_perf['nrmse'] = rand_clust_test_perf['rmse']/rand_clust_test_perf['mean']
rand_clust_test_perf['smae'] = rand_clust_test_perf['mae']/rand_clust_test_perf['mean']

In [40]:
rand_clust_test_perf.mean()

rmse      39.494879
mae       26.728996
mean     377.058594
nrmse      0.126755
smae       0.086209
dtype: float64

In [41]:
# Compute PIs with 1000 bootstrap samples
rand3_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust3,
                                            res_rand_clust_3, 
                                            nboot)


rand2_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust2,
                                            res_rand_clust_2, 
                                            nboot)

rand1_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust1,
                                            res_rand_clust_1, 
                                            nboot)

rand4_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_rand_clust4,
                                            res_rand_clust_4, 
                                            nboot)

In [42]:
rand_boot_ints = rand3_boot_ints.append(rand2_boot_ints).append(rand1_boot_ints).append(rand4_boot_ints)

In [43]:
# Add the true values into their own df column
rand_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [44]:
# Compute the 95% and 80% PI scores using the above function as new data frame columns
rand_boot_ints['int_95_score'] = interval_score(rand_boot_ints.actual, 
                                                    rand_boot_ints.lo_95,
                                                    rand_boot_ints.hi_95,
                                                    0.95)
                                                    
rand_boot_ints['int_80_score'] = interval_score(rand_boot_ints.actual, 
                                                    rand_boot_ints.lo_80,
                                                    rand_boot_ints.hi_80,
                                                    0.80)

In [45]:
rand_boot_ints.mean()

ts_index          2.500000
test_preds      374.865837
lo_95           321.235825
hi_95           430.300318
lo_80           347.598643
hi_80           403.010866
actual          377.058594
int_95_score    332.400729
int_80_score    167.309605
dtype: float64

In [46]:
rand_boot_ints_group = rand_boot_ints.groupby("ts_index").mean().reset_index()
rand_boot_ints_group['mean'] = rand_clust_test_perf['mean'].values
rand_boot_ints_group['int_95_score_scaled'] = rand_boot_ints_group['int_95_score']/rand_boot_ints_group['mean']
rand_boot_ints_group['int_80_score_scaled'] = rand_boot_ints_group['int_80_score']/rand_boot_ints_group['mean']

In [47]:
rand_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.902971
int_80_score_scaled    0.507723
dtype: float64

# Highway System

In [48]:
eng_mod = joblib.load("Results/Global/LightGBM Bayes/Highway System/model_0")

In [49]:
highway_res = pd.read_csv("Results/Global/LightGBM Bayes/Highway System/residual.csv")
highway_res['residual'] = highway_res['residual'].apply(eval)

eng_res = highway_res.query("cluster==1")['residual'].values[0]

In [50]:
unseen_test_preds_highway = compute_lgbm_test_preds(eng_mod, 
                                                    test_df_full,
                                                    lag_n)

In [51]:
highway_test_perf = compute_lgbm_test_perf(unseen_test_preds_highway, test_df_full)


In [52]:
highway_test_perf['nrmse'] = highway_test_perf['rmse']/highway_test_perf['mean']
highway_test_perf['smae'] = highway_test_perf['mae']/highway_test_perf['mean']

In [53]:
highway_test_perf.mean()

rmse      38.569299
mae       25.925766
mean     377.058594
nrmse      0.125855
smae       0.084882
dtype: float64

In [54]:
highway_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_highway,
                                              eng_res,
                                              nboot)

In [55]:
highway_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [56]:
highway_boot_ints['int_95_score'] = interval_score(highway_boot_ints.actual, 
                                                    highway_boot_ints.lo_95,
                                                    highway_boot_ints.hi_95,
                                                    0.95)
                                                    
highway_boot_ints['int_80_score'] = interval_score(highway_boot_ints.actual, 
                                                    highway_boot_ints.lo_80,
                                                    highway_boot_ints.hi_80,
                                                    0.80)

In [57]:
highway_boot_ints.mean()

ts_index          2.500000
test_preds      377.848202
lo_95           304.159550
hi_95           455.688278
lo_80           342.259909
hi_80           414.967104
actual          377.058594
int_95_score    287.655857
int_80_score    157.130132
dtype: float64

In [58]:
highway_boot_ints_group = highway_boot_ints.groupby("ts_index").mean().reset_index()
highway_boot_ints_group['mean'] = highway_test_perf['mean'].values
highway_boot_ints_group['int_95_score_scaled'] = highway_boot_ints_group['int_95_score']/highway_boot_ints_group['mean']
highway_boot_ints_group['int_80_score_scaled'] = highway_boot_ints_group['int_80_score']/highway_boot_ints_group['mean']

In [59]:
highway_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.931228
int_80_score_scaled    0.516865
dtype: float64

# Catch22 KMeans Clusters

In [60]:
unseen_sen_clust.catch22

0    1
1    1
2    4
3    4
Name: catch22, dtype: int64

In [61]:
catch22_mod1 = joblib.load("Results/Global/LightGBM Bayes/Catch22 KMeans/model_0")
catch22_mod4 = joblib.load("Results/Global/LightGBM Bayes/Catch22 KMeans/model_3")

In [62]:
catch22_res = pd.read_csv("Results/Global/LightGBM Bayes/Catch22 KMeans/residual.csv")
catch22_res['residual'] = catch22_res['residual'].apply(eval)

catch22_res1 = catch22_res.query("cluster==1")['residual'].values[0]
catch22_res4 = catch22_res.query("cluster==4")['residual'].values[0]

In [63]:
unseen_test_preds_catch22_clust1 = compute_lgbm_test_preds(catch22_mod1, 
                                                           test_df_full.query("ts_index in [1,2]"),
                                                           lag_n)

unseen_test_preds_catch22_clust4 = compute_lgbm_test_preds(catch22_mod4, 
                                                           test_df_full.query("ts_index in [3,4]"),
                                                           lag_n)

unseen_test_preds_catch22 = unseen_test_preds_catch22_clust1.append(unseen_test_preds_catch22_clust4)

In [64]:
catch22_test_perf = compute_lgbm_test_perf(unseen_test_preds_catch22, test_df_full)


In [65]:
catch22_test_perf['nrmse'] = catch22_test_perf['rmse']/catch22_test_perf['mean']
catch22_test_perf['smae'] = catch22_test_perf['mae']/catch22_test_perf['mean']

In [66]:
catch22_test_perf.mean()

rmse      38.981098
mae       26.282849
mean     377.058594
nrmse      0.124918
smae       0.084733
dtype: float64

In [67]:
catch22_boot_ints_clust1 = compute_lgbm_boostrap_int(unseen_test_preds_catch22_clust1,
                                                     catch22_res1,
                                                     nboot)

catch22_boot_ints_clust4 = compute_lgbm_boostrap_int(unseen_test_preds_catch22_clust4,
                                                     catch22_res4,
                                                     nboot)

catch22_boot_ints = catch22_boot_ints_clust1.append(catch22_boot_ints_clust4)

In [68]:
catch22_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [69]:
catch22_boot_ints['int_95_score'] = interval_score(catch22_boot_ints.actual, 
                                                    catch22_boot_ints.lo_95,
                                                    catch22_boot_ints.hi_95,
                                                    0.95)
                                                    
catch22_boot_ints['int_80_score'] = interval_score(catch22_boot_ints.actual, 
                                                    catch22_boot_ints.lo_80,
                                                    catch22_boot_ints.hi_80,
                                                    0.80)

In [70]:
catch22_boot_ints.mean()

ts_index          2.500000
test_preds      377.406667
lo_95           317.048464
hi_95           440.852390
lo_80           346.044561
hi_80           409.335424
actual          377.058594
int_95_score    284.131847
int_80_score    154.229379
dtype: float64

In [71]:
catch22_boot_ints_group = catch22_boot_ints.groupby("ts_index").mean().reset_index()
catch22_boot_ints_group['mean'] = catch22_test_perf['mean'].values
catch22_boot_ints_group['int_95_score_scaled'] = catch22_boot_ints_group['int_95_score']/catch22_boot_ints_group['mean']
catch22_boot_ints_group['int_80_score_scaled'] = catch22_boot_ints_group['int_80_score']/catch22_boot_ints_group['mean']

In [72]:
catch22_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.810287
int_80_score_scaled    0.478301
dtype: float64

# TSFeat KMeans Clusters

In [73]:
unseen_sen_clust.tsfeat

0    1
1    1
2    1
3    1
Name: tsfeat, dtype: int64

In [74]:
tsfeat_mod1 = joblib.load("Results/Global/LightGBM Bayes/TSFeat KMeans/model_0")

In [75]:
tsfeat_res = pd.read_csv("Results/Global/LightGBM Bayes/TSFeat KMeans/residual.csv")
tsfeat_res['residual'] = tsfeat_res['residual'].apply(eval)

tsfeat_res1 = tsfeat_res.query("cluster==1")['residual'].values[0]

In [76]:
unseen_test_preds_tsfeat1 = compute_lgbm_test_preds(tsfeat_mod1, 
                                                    test_df_full,
                                                    lag_n)

In [77]:
tsfeat_test_perf = compute_lgbm_test_perf(unseen_test_preds_tsfeat1, test_df_full)

In [78]:
tsfeat_test_perf['nrmse'] = tsfeat_test_perf['rmse']/tsfeat_test_perf['mean']
tsfeat_test_perf['smae'] = tsfeat_test_perf['mae']/tsfeat_test_perf['mean']

In [79]:
tsfeat_test_perf.mean()

rmse      38.460395
mae       25.785030
mean     377.058594
nrmse      0.125117
smae       0.084456
dtype: float64

In [80]:
tsfeat_boot_ints = compute_lgbm_boostrap_int(unseen_test_preds_tsfeat1,
                                             tsfeat_res1,
                                             nboot)

In [81]:
tsfeat_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [82]:
tsfeat_boot_ints['int_95_score'] = interval_score(tsfeat_boot_ints.actual, 
                                                    tsfeat_boot_ints.lo_95,
                                                    tsfeat_boot_ints.hi_95,
                                                    0.95)
                                                    
tsfeat_boot_ints['int_80_score'] = interval_score(tsfeat_boot_ints.actual, 
                                                    tsfeat_boot_ints.lo_80,
                                                    tsfeat_boot_ints.hi_80,
                                                    0.80)

In [83]:
tsfeat_boot_ints.mean()

ts_index          2.500000
test_preds      377.615412
lo_95           311.815116
hi_95           446.136497
lo_80           344.509566
hi_80           411.506543
actual          377.058594
int_95_score    298.728097
int_80_score    157.504571
dtype: float64

In [84]:
tsfeat_boot_ints_group = tsfeat_boot_ints.groupby("ts_index").mean().reset_index()
tsfeat_boot_ints_group['mean'] = tsfeat_test_perf['mean'].values
tsfeat_boot_ints_group['int_95_score_scaled'] = tsfeat_boot_ints_group['int_95_score']/tsfeat_boot_ints_group['mean']
tsfeat_boot_ints_group['int_80_score_scaled'] = tsfeat_boot_ints_group['int_80_score']/tsfeat_boot_ints_group['mean']

In [85]:
tsfeat_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.900324
int_80_score_scaled    0.505944
dtype: float64

# Train and Test - DTW Clusters

In [86]:
unseen_sen_clust.dtw

0    2
1    2
2    1
3    1
Name: dtw, dtype: int64

In [87]:
dtw_mod2 = joblib.load("Results/Global/LightGBM Bayes/DTW/model_1")
dtw_mod1 = joblib.load("Results/Global/LightGBM Bayes/DTW/model_0")

In [88]:
dtw_res = pd.read_csv("Results/Global/LightGBM Bayes/DTW/residual.csv")
dtw_res['residual'] = dtw_res['residual'].apply(eval)

dtw_res2 = dtw_res.query("cluster==2")['residual'].values[0]
dtw_res1 = dtw_res.query("cluster==1")['residual'].values[0]

In [89]:
unseen_test_preds_dtw2 = compute_lgbm_test_preds(dtw_mod2, 
                                                 test_df_full.query("ts_index in [1,2]"),
                                                 lag_n)

unseen_test_preds_dtw1 = compute_lgbm_test_preds(dtw_mod1, 
                                                 test_df_full.query("ts_index in [3,4]"),
                                                 lag_n)

In [90]:
dtw_test_perf2 = compute_lgbm_test_perf(unseen_test_preds_dtw2, test_df_full.query("ts_index in [1,2]"))

dtw_test_perf1 = compute_lgbm_test_perf(unseen_test_preds_dtw1, test_df_full.query("ts_index in [3,4]"))

dtw_test_perf = dtw_test_perf2.append(dtw_test_perf1)

In [91]:
dtw_test_perf['nrmse'] = dtw_test_perf['rmse']/dtw_test_perf['mean']
dtw_test_perf['smae'] = dtw_test_perf['mae']/dtw_test_perf['mean']

In [92]:
dtw_test_perf.mean()

rmse      37.798783
mae       25.323766
mean     377.058594
nrmse      0.121270
smae       0.081587
dtype: float64

In [93]:
dtw_boot_ints2 = compute_lgbm_boostrap_int(unseen_test_preds_dtw2,
                                           dtw_res2,
                                           nboot)

dtw_boot_ints1 = compute_lgbm_boostrap_int(unseen_test_preds_dtw1,
                                           dtw_res1,
                                           nboot)

In [94]:
dtw_boot_ints = dtw_boot_ints2.append(dtw_boot_ints1)
dtw_boot_ints['actual'] = test_df_full.iloc[:,0].to_list()

In [95]:
dtw_boot_ints['int_95_score'] = interval_score(dtw_boot_ints.actual, 
                                               dtw_boot_ints.lo_95,
                                               dtw_boot_ints.hi_95,
                                               0.95)
                                                    
dtw_boot_ints['int_80_score'] = interval_score(dtw_boot_ints.actual, 
                                               dtw_boot_ints.lo_80,
                                               dtw_boot_ints.hi_80,
                                               0.80)

In [96]:
dtw_boot_ints.mean()

ts_index          2.500000
test_preds      376.987037
lo_95           318.666677
hi_95           437.309461
lo_80           345.554877
hi_80           409.303636
actual          377.058594
int_95_score    260.576473
int_80_score    144.497565
dtype: float64

In [97]:
dtw_boot_ints_group = dtw_boot_ints.groupby("ts_index").mean().reset_index()
dtw_boot_ints_group['mean'] = dtw_test_perf['mean'].values
dtw_boot_ints_group['int_95_score_scaled'] = dtw_boot_ints_group['int_95_score']/dtw_boot_ints_group['mean']
dtw_boot_ints_group['int_80_score_scaled'] = dtw_boot_ints_group['int_80_score']/dtw_boot_ints_group['mean']

In [98]:
dtw_boot_ints_group[['int_95_score_scaled', 'int_80_score_scaled']].mean()

int_95_score_scaled    0.771050
int_80_score_scaled    0.455168
dtype: float64